In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [82]:
# Taking input and creating train test splits
data = pd.read_csv('spam.csv',encoding = 'latin-1')
data = data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'], axis=1)
data = np.asarray(data)
split_ratio = 0.7
split = int(split_ratio * data.shape[0])
data_train = data[:split+1, :]
data_test = data[split+1:,:]
print(len(data))

5572


In [73]:
#Data Preprocessing
def Preprocessing(mail):
    mail = mail.lower()
    mail = word_tokenize(mail)
    stopword = stopwords.words('english')
    words = []
    for word in mail:
        if word not in stopword and len(word) > 2: 
            words.append(word)
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    return words

In [81]:
#String counts for spam and ham words
message_train = []
for data in data_train:
    words = Preprocessing(data[1])
    message_train.append(words)
spam = {}
ham = {}
all_words = {}
spam_word_ct = 0
ham_word_ct = 0
total_words = 0
spam_ct = 0
ham_ct = 0

for i,message in enumerate(message_train):
    if data_train[i][0] == 'spam':
        spam_ct += 1
        for word in message:
            if word not in all_words:
                all_words[word] = 1
            else:
                all_words[word] += 1
            total_words += 1
            spam_word_ct += 1
            if word not in spam:
                spam[word] = 1
            else:
                spam[word] += 1
    if data_train[i][0] == 'ham':
        ham_ct += 1
        for word in message:
            if word not in all_words:
                all_words[word] = 1
            else:
                all_words[word] += 1
            total_words += 1
            ham_word_ct += 1
            if word not in ham:
                ham[word] = 1
            else:
                ham[word] += 1
                
print(len(all_words))
print(len(spam))
print(len(ham))
print(spam_ct, ham_ct, spam_ct+ham_ct)

6674
2176
5171
519 3382 3901


In [68]:
def Predict(mail):
    probaility_spam = spam_ct / len(data_train)
    probaility_ham = ham_ct / len(data_train)
    words = Preprocessing(mail)
    for word in words:
        if word in spam:
            probaility_spam *= (spam[word] / spam_word_ct) / (all_words[word] / total_words)
        else:
            #Laplacian smoothening with alpha = 1
            probaility_spam *= (1 / (spam_word_ct+len(spam)))
            if word in all_words:
                probaility_spam /= (all_words[word] / total_words)
            else:
                probaility_spam /= (1 / (total_words+len(all_words)))
        if word in ham:
            probaility_ham *= (ham[word] / ham_word_ct) / (all_words[word] / total_words)
        else:
            probaility_ham *= (1 / (ham_word_ct+len(ham)))
            if word in all_words:
                probaility_ham /= (all_words[word] / total_words)
            else:
                probaility_ham /= (1 / (total_words+len(all_words)))
    if probaility_spam >= probaility_ham:
        return 'spam'
    else:
        return 'ham'

In [69]:
# Testing
confusion_matrix = np.zeros((2,2))
for data in data_test:
    predictions.append(Predict(data[1]))
    if data[0] == 'ham':
        if Predict(data[1]) == 'ham':
            confusion_matrix[0][0] += 1
        else:
            confusion_matrix[0][1] += 1
    else:
        if Predict(data[1]) == 'spam':
            confusion_matrix[1][1] += 1
        else:
            confusion_matrix[1][0] += 1
print('Confusion Matrix')
print(confusion_matrix)
print('Accuracy Percentage : ', (confusion_matrix[0][0]+ confusion_matrix[1][1]) / len(data_test) * 100)

Confusion Matrix
[[1350.   93.]
 [   4.  224.]]
Accuracy Percentage :  94.19509275882704
